In [1]:
import mne #for exploring,visualizing and analyzing neurophysiological data such as EEG
import pandas as pd #data analysis and manipulation tool
import os  #for interacting with operating system, os.path to interact with file system
import numpy as np
import datetime
import json #Javascript Object Notation.lightweight format for storing and transporting data
from sklearn.preprocessing import MinMaxScaler #useful to transform features by scaling each feature to a given range
import re # the functions in this module let you check if a particular string matches a given regular expression
          ##a regular expression is a sequence of characters, these characters are to be searched in a text

import seaborn as sns #data visualization library based on matplotlib
import matplotlib.pyplot as plt
import shap #used to explain output of machine learning model using Shapley values. an application of game theory
import xgboost as xgb #machine learning library for regression, classification and ranking problems

In [2]:
def read_eeg(root): #function to retreive data from vhdr file. This function is called later. the input arguement is in the cell
                    ##cell consisting the trials array.  
                    
    vhdr_fname=root+trial_day.split("/")[0]+"_"+trial_num.split("/")[0]+"_EEG01.vhdr" #the split function removes the / from the
    ##strings in the trails array. trial_day and trial_num is defined in the cell below trials.
                                 
    #define list of indices for non-eeg channels
    misc_list=[]               #there are in total 82 channels. 64EEG channels +18 non EEG channels
    for i in range(18):
        misc_list.append(i+64)
        
        
    #read raw data #read_raw_brainvision--->reader for brain vision EEG file
    raw=mne.io.read_raw_brainvision(vhdr_fname,misc=misc_list,preload=True,verbose=False) 
    raw.info['line_freq']=500. #signals a float number
    
    #set montage
    montage=mne.channels.make_standard_montage('easycap-M1') #name of montage to use
    raw.set_montage(montage,verbose=False)
    
    
    #set common average referance,to specify the desired referance for EEG. this can be an existing electrode or virtual channel
    raw.set_eeg_reference('average', projection=False,verbose=False)
    
    #create a pandas dataframe with eeg data
    eeg_data = pd.DataFrame(raw.get_data().transpose(), columns=raw.ch_names)
    
    # Temporarily drop T7 while it's broken
    #eeg_data = eeg_data.drop(columns=["T7"])
    
    return eeg_data

In [3]:
def eeg_times(root):
    
    vhdr_path = root + trial_day.split("/")[0] + "_" + trial_num.split("/")[0] + "_EEG01.vhdr"
    
    import re

    with open(vhdr_path, "r+") as file: #r+ is used for both reading and writing. no need to call file.close() with the
                                        ##with statement. with statement ensures proper acquisition and release of resources

       #read the file contents
       file_contents = file.read()  #reads all the contents of the file
       text_pattern = re.compile(re.escape("%")) # the term escape is used to make % ineffective and compile is to compile a 
                                                 # a regular expression
       file_contents = text_pattern.sub("", file_contents) #substitutes "" in te file contents
       file.seek(0) #changes the pointer or lets say location in the file
       file.truncate() #truncates file from the current index. which in this case is zero.
       file.write(file_contents) 
    

    with open(vhdr_path, "r") as r:
        vhdr_lines = r.readlines() #reads all the lines
    
    eeg_start = ""

    for i,j in enumerate(vhdr_lines):
        if 'Recording Start Time' in j:
            eeg_start = vhdr_lines[i+1]
    eeg_start = eeg_start.splitlines()[0] #.split(".")[0]

    eeg_start_dt = datetime.datetime.strptime(eeg_start,'%Y-%m-%d %H:%M:%S.%f')

    return eeg_start_dt

In [4]:
def tobii_times(root): #the function is to read tobii data i.e. the pupilometric data. Here only the start and end datetime
                       ##is extracted

    with open(root + 'segment.json','r') as f: 
        seg_data = json.load(f)

    seg_start = seg_data['seg_t_start']
    seg_end = seg_data['seg_t_stop']

    seg_start = seg_start.replace("T"," ").replace("+",".")
    seg_start_dt = datetime.datetime.strptime(seg_start, '%Y-%m-%d %H:%M:%S.%f')

    seg_end = seg_end.replace("T"," ").replace("+",".")
    seg_end_dt = datetime.datetime.strptime(seg_end, '%Y-%m-%d %H:%M:%S.%f')
    
    return seg_start_dt, seg_end_dt

In [5]:
def EEG_start_sync(tobii_start, tobii_end, eeg_start, eeg_df):    #syncing the start time betwween the tobii start 
                                                                  ##and eeg start because eeg starts earlier        
        #eeg_start in the eeg_times() function
        #eeg_df, tobii_start and tobii_end is in the same cell as where root is defined. below the trial cell
    fs = 500    
    skip_eeg_data = round((tobii_start - eeg_start).total_seconds()*fs) # 500 is the frequency

    total_event_duration = (tobii_end - tobii_start).total_seconds()
    event_eeg_duration = total_event_duration*fs

    eeg_data = eeg_df.iloc[skip_eeg_data:int(skip_eeg_data + event_eeg_duration)].reset_index().drop('index', axis=1)
    eeg_data['dateTime'] = pd.date_range(start=tobii_start, end=tobii_end, periods=len(eeg_data))
    eeg_data.index = eeg_data['dateTime']
    return eeg_data

In [6]:
def getEyeData(lines, tobii_start, tobii_end):  # function to read the tobii data
    gd, pc, pdi, gp, gp3 = [[],[]],[[],[]],[[],[]],[],[]
    ts, status = [], []
    j = 0
    for i in lines:
        # if i['s'] == 0:
        if 'gidx' in i:
            ts.append(i['ts'])
            j += 1
            if 'eye' in i:
                if i['eye'] == 'left':
                    if 'gd' in i:
                        gd[0].append(i['gd'])
                    elif 'pc' in i:
                        pc[0].append(i['pc'])
                    elif 'pd' in i:
                        pdi[0].append(i['pd'])
                        if i['s'] == 0:
                            status.append(i['s'])
                        elif i['s'] != 0:
                            status.append(i['s'])     
                elif i['eye'] == 'right':
                    if 'gd' in i:
                        gd[1].append(i['gd'])
                    elif 'pc' in i:
                        pc[1].append(i['pc'])
                    elif 'pd' in i:
                        pdi[1].append(i['pd'])
            elif 'gp' in i:
                gp.append(i['gp'])
            elif 'gp3' in i:
                gp3.append(i['gp3'])
        # elif i['s'] != 0:
        #         continue
    
    gd_df_left = pd.DataFrame(gd[0])
    gd_df_right = pd.DataFrame(gd[1])
    pc_df_left = pd.DataFrame(pc[0])
    pc_df_right = pd.DataFrame(pc[1])
    pdi_df_left = pd.DataFrame(pdi[0])
    pdi_df_right = pd.DataFrame(pdi[1])
    pda = (pdi_df_left + pdi_df_right)/2
    pd_diff = pdi_df_left - pdi_df_right
    pd_diff_abs = abs(pdi_df_left - pdi_df_right)
    pup_dis = pc_df_left.iloc[:,0] - pc_df_right.iloc[:,0]
    gp_df = pd.DataFrame(gp)
    gp3_df = pd.DataFrame(gp3)
    ts_np = np.unique(np.array(ts))
    ts = pd.DataFrame(ts_np)
    time = pd.DataFrame(pd.date_range(start=tobii_start, end=tobii_end, periods=len(ts)))
    status_df = pd.DataFrame(status)
    # print(status_df)
    
    
    df_full = pd.concat([time, status_df, gd_df_left, gd_df_right, pc_df_left, pc_df_right, pdi_df_left, pdi_df_right, pda, gp_df, gp3_df, pup_dis, pd_diff, pd_diff_abs], axis=1, ignore_index=True)
    df_full.columns = ['dateTime','status_tobii', 'gd_lx', 'gd_ly', 'gd_lz', 'gd_rx', 'gd_ry', 'gd_rz', 'pc_lx', 'pc_ly', 'pc_lz', 'pc_rx', 'pc_ry', 'pc_rz', 'pd_l', 'pd_r', 
                       'pda','gp_x', 'gp_y', 'gp3_x','gp3_y','gp3_z','pup_dis', 'pd_diff', 'pd_diff_abs']

    
    return df_full

In [7]:
def readTobii(root, tobii_start, tobii_end):
    lines = []
    # tobii_data = root + 'livedata.json/livedata.json'
    for line in open(root + 'livedata.json/livedata.json', 'r'):
        lines.append(json.loads(line))

    eye_metrics = getEyeData(lines, tobii_start, tobii_end)
    eye_metrics = eye_metrics.set_index('dateTime')
    # clean_em = eye_metrics[eye_metrics['status'] == 0]
    
#     eye_ratio = pd.read_csv(root + "eyesstream.csv")
#     eye_ratio['dateTime'] = pd.date_range(start=tobii_start, end=tobii_end, periods=len(eye_ratio))
#     eye_ratio = eye_ratio.loc[:,['dateTime','ratio']].set_index('dateTime')
    
#     # sync eye_ratio and eye_metrics
#     eye_combined = pd.merge_asof(left=eye_ratio, right=eye_metrics, right_index=True, left_index=True, direction='nearest', tolerance=pd.Timedelta('0.02s'))
    
#     return eye_combined
    return eye_metrics

In [8]:
def getSensorData(root, sensor, start, end, location=None):
    
    # location is roof or trunk
    # sensor is the name of sensor
    trial_day_roof = trial_day + "data/roofTopMints/" + trial_day.split("_")[0] + "/" + trial_day.split("_")[1] + "/" + trial_day.split("_")[2]
    trial_day_trunk = trial_day + "data/trunkSensors/" + trial_day.split("_")[0] + "/" + trial_day.split("_")[1] + "/" + trial_day.split("_")[2]

    if location == "roof":
        df = pd.read_csv(trial_day_roof + "MINTS_001e0636e527_" + sensor + "_" + root.split("/")[0] + ".csv")
    elif location == "trunk":
        df = pd.read_excel(trial_day_trunk + "MINTS_001e0610c2e7_" + sensor + "_" + root.split("/")[0] + ".xlsx", index_col=None)
    elif sensor == "palas":
        df = pd.read_csv(trial_day + trial_day.split("/")[0] + "_all_bins.csv")
    
    df['dateTime'] = pd.to_datetime(df['dateTime'])
    df = df.loc[(df['dateTime'] > start) & (df['dateTime'] < end)].reset_index().drop('index', axis=1).set_index('dateTime')
                         
    return df

In [9]:
def scaleSensor(df,var,roll_window=None, aggregation=None):
    
    scale = MinMaxScaler() 
    dummy = pd.DataFrame()
    if roll_window and aggregation == "mean":
        dummy[var] = df[var].rolling(roll_window).mean()
    elif roll_window and aggregation == "sum":
        dummy[var] = df[var].rolling(roll_window).sum()
    else:
        dummy[var] = df[var]
    return scale.fit_transform(dummy[var].to_numpy().reshape(-1,1))

In [10]:
def syncSensorTobiiEEG(sensor_df, eyes_df, eeg_df):
    
    tol_1 = pd.Timedelta('0.02 second') # 100 Hz (or whatever that is min(eeg_HZ, eyes_HZ))
    eye_eeg = pd.merge_asof(left=eyes_df, right=eeg_df, right_index=True, left_index=True, direction='nearest', tolerance=tol_1)
    
    tol_2 = pd.Timedelta('1S')
    sensor_eye_eeg = pd.merge_asof(left=sensor_df, right=eye_eeg, right_index=True, left_index=True, direction='nearest', tolerance=tol_2).dropna()
    # print(list(sensor_eye_eeg.columns))
    sensor_eye_eeg = sensor_eye_eeg[sensor_eye_eeg['status_tobii'] == 0]
    sensor_eye_eeg = sensor_eye_eeg[sensor_eye_eeg['pc_rx'] != 0]
    
    return sensor_eye_eeg

In [11]:
trials = [
    ["2021_06_09/", "T03_U000/"],
    ["2021_06_09/", "T04_U000/"],
    ["2021_06_10/", "T03_U000/"],
    ["2021_06_10/", "T04_U000/"],
]


In [12]:
%%time

full_df = pd.DataFrame()

for i in trials:
    trial_day, trial_num, root = i[0], i[1], i[0] + i[1]
    eeg_start = eeg_times(root) 
    tobii_start, tobii_end = tobii_times(root)
    eeg_df_full = read_eeg(root)   #Will retrieve the eeg data
    eeg_df = EEG_start_sync(tobii_start, tobii_end, eeg_start, eeg_df_full)
    tobii_data = readTobii(root, tobii_start, tobii_end, ) #Will retreive the tobii data
    
    # sensors
    
    # sensor_df = getSensorData(root, 'MGS001', tobii_start, tobii_end, 'roof')
    #sensor_df = getSensorData(root, 'palas', tobii_start, tobii_end)
    # print("Parsing " + root)
    sensor_df = getSensorData(root, 'CO', tobii_start, tobii_end, 'trunk')
    
    synced_full_df = syncSensorTobiiEEG(sensor_df, tobii_data, eeg_df)
    full_df = pd.concat([full_df,synced_full_df])
    

Wall time: 44.4 s


In [13]:
pd.set_option('display.max_columns', None)
eeg_df

,Fp1,Fp2,F3,F4,C3,C4,P3,P4,O1,O2,F7,F8,T7,T8,P7,P8,Fz,Cz,Pz,Oz,FC1,FC2,CP1,CP2,FC5,FC6,CP5,CP6,FT9,FT10,FCz,AFz,F1,F2,C1,C2,P1,P2,AF3,AF4,FC3,FC4,CP3,CP4,PO3,PO4,F5,F6,C5,C6,P5,P6,AF7,AF8,FT7,FT8,TP7,TP8,PO7,PO8,Fpz,CPz,POz,TP10,ExG 1,ExG 2,ExG 3,ExG 4,ECG.,Resp.,PPG,SpO2,HR,GSR,Temp.,AUX 1,AUX 2,ACC77,ACC78,ACC79,Packet Counter,TRIGGER,dateTime
dateTime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-06-10 11:21:49.000000000,0.001702,0.004508,-0.008591,-0.004093,-0.000593,0.002736,0.006220,0.004985,0.003791,0.000327,0.015805,-0.000413,0.001278,-0.001994,-0.006885,-0.002444,-0.000833,0.002420,-0.002364,-0.000941,0.002599,0.000066,-0.003732,0.008034,-0.002318,-0.000082,-0.002074,-0.001163,0.002574,-0.000088,0.003863,-0.002251,-0.002338,-0.002045,0.002106,0.000353,0.002081,0.004246,0.000054,0.002104,-0.008147,-0.002191,0.003456,0.004849,0.000031,0.003599,-0.000791,0.000939,0.001964,0.000076,0.000702,-0.001102,-0.001083,0.002184,-0.002757,-0.005135,-0.010631,-0.004571,-0.002086,-0.001366,0.000614,-0.000393,-0.004239,-0.000531,-0.052083,-0.052083,-0.052083,-0.052083,0.000809,35.716936,15856.999904,0.000098,96.0,11666.615486,29.7500,-0.041909,-0.070300,-4073.660851,10860.770226,-3843.471050,19853.0,0.0,2021-06-10 11:21:49.000000000
2021-06-10 11:21:49.002000012,0.001698,0.004494,-0.008588,-0.004090,-0.000590,0.002733,0.006221,0.004986,0.003783,0.000322,0.015812,-0.000423,0.001281,-0.001976,-0.006887,-0.002439,-0.000832,0.002422,-0.002364,-0.000951,0.002603,0.000069,-0.003728,0.008033,-0.002315,-0.000088,-0.002068,-0.001162,0.002573,-0.000073,0.003865,-0.002252,-0.002335,-0.002041,0.002109,0.000354,0.002082,0.004244,0.000055,0.002113,-0.008145,-0.002195,0.003459,0.004849,0.000026,0.003597,-0.000789,0.000946,0.001968,0.000070,0.000702,-0.001100,-0.001089,0.002144,-0.002754,-0.005147,-0.010635,-0.004556,-0.002090,-0.001366,0.000615,-0.000391,-0.004243,-0.000526,-0.052083,-0.052083,-0.052083,-0.052083,0.000833,35.716936,15856.999904,0.000098,96.0,11666.615486,29.7500,-0.041881,-0.070267,-4073.660851,10825.892448,-3822.544575,19854.0,0.0,2021-06-10 11:21:49.002000012
2021-06-10 11:21:49.004000024,0.001694,0.004482,-0.008588,-0.004090,-0.000589,0.002735,0.006220,0.004988,0.003780,0.000324,0.015813,-0.000435,0.001272,-0.001961,-0.006888,-0.002432,-0.000832,0.002425,-0.002363,-0.000950,0.002604,0.000070,-0.003725,0.008035,-0.002314,-0.000089,-0.002067,-0.001157,0.002583,-0.000084,0.003866,-0.002254,-0.002332,-0.002041,0.002109,0.000356,0.002082,0.004243,0.000053,0.002118,-0.008145,-0.002196,0.003461,0.004852,0.000024,0.003597,-0.000788,0.000938,0.001969,0.000070,0.000700,-0.001096,-0.001097,0.002125,-0.002752,-0.005142,-0.010643,-0.004547,-0.002093,-0.001362,0.000615,-0.000388,-0.004245,-0.000514,-0.052083,-0.052083,-0.052083,-0.052083,0.000877,35.716936,15856.999904,0.000098,96.0,11666.615486,29.7500,-0.041881,-0.070276,-4164.341450,10832.868576,-3822.544575,19855.0,0.0,2021-06-10 11:21:49.004000024
2021-06-10 11:21:49.006000036,0.001691,0.004489,-0.008589,-0.004097,-0.000592,0.002738,0.006217,0.004988,0.003786,0.000327,0.015808,-0.000442,0.001270,-0.001935,-0.006889,-0.002428,-0.000833,0.002429,-0.002364,-0.000947,0.002603,0.000069,-0.003726,0.008036,-0.002313,-0.000087,-0.002072,-0.001153,0.002568,-0.000092,0.003864,-0.002253,-0.002333,-0.002043,0.002108,0.000356,0.002081,0.004241,0.000051,0.002103,-0.008149,-0.002195,0.003459,0.004854,0.000025,0.003600,-0.000790,0.000931,0.001967,0.000072,0.000697,-0.001095,-0.001095,0.002154,-0.002754,-0.005130,-0.010649,-0.004548,-0.002088,-0.001357,0.000611,-0.000390,-0.004246,-0.000517,-0.052083,-0.052083,-0.052083,-0.052083,0.000912,35.716936,15856.999904,0.000098,96.0,11666.615486,29.7500,-0.041924,-0.070262,-4171.317101,10867.745399,-3829.519987,19856.0,0.0,2021-06-10 11:21:49.006000036
2021-06-10 11:21:49.008000049,0.001693,0.004494,-0.008589,-0.004100,-0.000593,0.002740,0.006213,0.004986,0.003780,0.000325,0.015805,-0.000435,0.001280,-0.001931,-0.006891,-

In [14]:
tobii_data

,status_tobii,gd_lx,gd_ly,gd_lz,gd_rx,gd_ry,gd_rz,pc_lx,pc_ly,pc_lz,pc_rx,pc_ry,pc_rz,pd_l,pd_r,pda,gp_x,gp_y,gp3_x,gp3_y,gp3_z,pup_dis,pd_diff,pd_diff_abs
dateTime,,,,,,,,,,,,,,,,,,,,,,,,
2021-06-10 11:21:49.000000000,0,0.0089,0.2535,0.9673,0.0601,0.2419,0.9684,31.18,-32.46,-32.70,-34.76,-30.32,-29.89,3.06,2.99,3.025,0.4831,0.2434,40.74,273.85,1161.22,65.94,0.07,0.07
2021-06-10 11:21:49.009975138,0,0.0112,0.2638,0.9645,0.0568,0.2527,0.9659,31.19,-32.68,-32.67,-34.82,-30.12,-29.94,3.10,3.03,3.065,0.4834,0.2286,44.95,324.11,1297.45,66.01,0.07,0.07
2021-06-10 11:21:49.019950277,0,0.0100,0.2627,0.9648,0.0576,0.2600,0.9639,31.17,-32.52,-32.66,-34.75,-30.36,-29.82,3.10,3.04,3.070,0.4835,0.2247,44.97,330.25,1303.15,65.92,0.06,0.06
2021-06-10 11:21:49.029925416,0,0.0115,0.2682,0.9633,0.0571,0.2613,0.9636,31.20,-32.72,-32.57,-34.82,-30.19,-29.80,3.09,3.02,3.055,0.4832,0.2202,47.05,345.25,1339.60,66.02,0.07,0.07
2021-06-10 11:21:49.039900555,0,0.0127,0.2708,0.9626,0.0566,0.2681,0.9617,31.24,-32.60,-32.78,-34.75,-30.41,-29.75,3.13,3.05,3.090,0.4829,0.2136,50.34,373.35,1414.36,65.99,0.08,0.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-10 11:27:13.960099444,1,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000,0.0000,0.0000,0.00,0.00,0.00,0.00,0.00,0.00
2021-06-10 11:27:13.970074583,1,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000,0.0000,0.0000,0.00,0.00,0.00,0.00,0.00,0.00
2021-06-10 11:27:13.980049722,1,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000,0.0000,0.0000,0.00,0.00,0.00,0.00,0.00,0.00


In [15]:
synced_full_df

,CO2,status_tobii,gd_lx,gd_ly,gd_lz,gd_rx,gd_ry,gd_rz,pc_lx,pc_ly,pc_lz,pc_rx,pc_ry,pc_rz,pd_l,pd_r,pda,gp_x,gp_y,gp3_x,gp3_y,gp3_z,pup_dis,pd_diff,pd_diff_abs,Fp1,Fp2,F3,F4,C3,C4,P3,P4,O1,O2,F7,F8,T7,T8,P7,P8,Fz,Cz,Pz,Oz,FC1,FC2,CP1,CP2,FC5,FC6,CP5,CP6,FT9,FT10,FCz,AFz,F1,F2,C1,C2,P1,P2,AF3,AF4,FC3,FC4,CP3,CP4,PO3,PO4,F5,F6,C5,C6,P5,P6,AF7,AF8,FT7,FT8,TP7,TP8,PO7,PO8,Fpz,CPz,POz,TP10,ExG 1,ExG 2,ExG 3,ExG 4,ECG.,Resp.,PPG,SpO2,HR,GSR,Temp.,AUX 1,AUX 2,ACC77,ACC78,ACC79,Packet Counter,TRIGGER,dateTime
dateTime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-06-10 11:21:49.679,768.470,0,-0.0019,0.2679,0.9634,0.0564,0.2730,0.9604,31.14,-32.71,-32.66,-34.78,-30.58,-29.34,3.01,2.95,2.980,0.4878,0.2186,28.62,270.78,1044.50,65.92,0.06,0.06,0.001684,0.004439,-0.008507,-0.003935,-0.000692,0.002632,0.006128,0.004929,0.003817,0.000305,0.015819,-0.000286,0.001311,-0.001800,-0.006851,-0.002334,-0.000918,0.002350,-0.002432,-0.000933,0.002498,-0.000017,-0.003832,0.007907,-0.002317,-0.000073,-0.002125,-0.001087,0.002643,-0.000006,0.003771,-0.002162,-0.002325,-0.001985,0.002016,0.000264,0.002012,0.004167,0.000072,0.002135,-0.008190,-0.002253,0.003336,0.004769,-0.000024,0.003660,-0.000757,0.001003,0.001980,0.000161,0.000719,-0.000921,-0.001143,0.002152,-0.002687,-0.004929,-0.010624,-0.004496,-0.002050,-0.001275,0.000559,-0.000549,-0.004268,-0.000454,-0.052083,-0.052083,-0.052083,-0.052083,0.000871,36.037885,15967.000276,0.000098,97.0,11691.340446,29.8125,-0.042148,-0.070243,-4450.335026,11014.229774,-3934.151888,20192.0,0.0,2021-06-10 11:21:49.678004172
2021-06-10 11:21:50.678,771.514,0,0.0371,0.0168,0.9992,0.1033,-0.0009,0.9946,31.32,-33.84,-32.70,-34.40,-31.71,-29.22,3.16,3.13,3.145,0.4608,0.5219,64.07,-25.37,900.53,65.72,0.03,0.03,0.001691,0.004453,-0.008585,-0.004190,-0.000635,0.002711,0.006207,0.005051,0.003820,0.000280,0.015808,-0.000614,0.001288,-0.001884,-0.006890,-0.002400,-0.000841,0.002414,-0.002361,-0.000935,0.002546,0.000026,-0.003789,0.007945,-0.002264,-0.000035,-0.002062,-0.001022,0.002509,-0.000341,0.003817,-0.002163,-0.002261,-0.002003,0.002071,0.000352,0.002089,0.004250,0.000094,0.002091,-0.008134,-0.002225,0.003396,0.004889,0.000078,0.003696,-0.000765,0.000737,0.002035,0.000202,0.000797,-0.000867,-0.001173,0.002149,-0.002682,-0.005057,-0.010682,-0.004525,-0.002039,-0.001317,0.000555,-0.000525,-0.004226,-0.000553,-0.052083,-0.052083,-0.052083,-0.052083,0.000573,35.946187,16310.999170,0.000098,96.0,11677.211761,29.6875,-0.042090,-0.070329,-4129.464149,10721.261024,-3982.980013,20691.0,0.0,2021-06-10 11:21:50.676010313
2021-06-10 11:21:51.679,771.288,0,0.1481,-0.1399,0.9790,0.1850,-0.1594,0.9697,31.53,-34.81,-32.35,-33.68,-32.30,-29.27,3.16,3.16,3.160,0.3988,0.6860,225.00,-236.71,1291.43,65.21,0.00,0.00,0.001782,0.004579,-0.008592,-0.004178,-0.000591,0.002703,0.006241,0.005018,0.003771,0.000246,0.015898,-0.000519,0.001266,-0.001921,-0.006899,-0.002531,-0.000780,0.002417,-0.002415,-0.000941,0.002574,0.000028,-0.003794,0.007932,-0.002258,-0.000113,-0.002022,-0.001104,0.002548,-0.000290,0.003870,-0.002152,-0.002298,-0.002062,0.002122,0.000355,0.002061,0.004193,0.000136,0.002167,-0.008136,-0.002253,0.003451,0.004874,0.000047,0.003597,-0.000694,0.000828,0.002020,0.000094,0.000782,-0.001001,-0.000987,0.002306,-0.002657,-0.005125,-0.010696,-0.004590,-0.002105,-0.001416,0.000673,-0.000568,-0.004333,-0.000560,-0.052083,-0.052083,-0.052083,-0.052083,0.001175,36.358833,16731.999815,0.000098,96.0,11684.276581,29.7500,-0.042086,-0.070343,-1513.671875,10777.064323,-2476.283550,21195.0,0.0,2021-06-10 11:21:51.684016517
2021-06-10 11:21:52.678,771.694,0,0.0161,-0.3654,0.9307,0.1061,-0.3429,0.9334,30.61,-36.69,-30.27,-34.56,-34.09,-28.20,3.42,3.26,3.340,0.4626,0.9817,39.25,-273.94,598.71,65.17,0.16,0.16,0.001670,0.004473,-0.008609,-0.003970,-0.000645,0.002622,0.006179,0.004973,0.003777,0.000288,0.015932,-0.000168,0.001294,-0.001778,-0.006863,-0.002465,-0.000863,0.002318,-0.002514,

In [16]:
pd.set_option('display.max_columns', None)
full_df

,CO2,status_tobii,gd_lx,gd_ly,gd_lz,gd_rx,gd_ry,gd_rz,pc_lx,pc_ly,pc_lz,pc_rx,pc_ry,pc_rz,pd_l,pd_r,pda,gp_x,gp_y,gp3_x,gp3_y,gp3_z,pup_dis,pd_diff,pd_diff_abs,Fp1,Fp2,F3,F4,C3,C4,P3,P4,O1,O2,F7,F8,T7,T8,P7,P8,Fz,Cz,Pz,Oz,FC1,FC2,CP1,CP2,FC5,FC6,CP5,CP6,FT9,FT10,FCz,AFz,F1,F2,C1,C2,P1,P2,AF3,AF4,FC3,FC4,CP3,CP4,PO3,PO4,F5,F6,C5,C6,P5,P6,AF7,AF8,FT7,FT8,TP7,TP8,PO7,PO8,Fpz,CPz,POz,TP10,ExG 1,ExG 2,ExG 3,ExG 4,ECG.,Resp.,PPG,SpO2,HR,GSR,Temp.,AUX 1,AUX 2,ACC77,ACC78,ACC79,Packet Counter,TRIGGER,dateTime
dateTime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-06-09 10:55:43.047,347.151,0.0,-0.0824,0.2385,0.9676,-0.0080,0.2476,0.9688,30.55,-31.32,-32.70,-35.17,-31.16,-29.38,4.77,4.77,4.770,0.5346,0.2579,-41.70,179.94,810.24,65.72,0.00,0.00,0.002513,0.000333,-0.004642,-0.000896,0.003327,-0.004718,0.007154,0.003981,0.000949,-0.005878,0.000579,0.002672,0.005977,-0.000076,-0.004688,-0.006041,0.003246,0.006336,-0.002376,0.000365,0.001624,0.004367,0.000338,0.006253,0.000400,-0.003808,-0.000728,-0.004377,-0.005192,0.001358,0.006076,0.002321,-0.004135,-0.000339,0.004509,0.003106,0.007275,0.003379,-0.004735,-0.004000,-0.002873,0.000161,0.004933,0.004254,0.003954,0.002995,-0.003353,-0.001476,0.001975,-0.000322,0.001375,-0.001364,-0.002911,-0.000561,0.001138,-0.002866,-0.005902,-0.004078,-0.000808,-0.006510,-0.001011,-0.005793,0.002933,-0.005697,-0.052083,-0.052083,-0.052083,-0.052083,-0.002129,36.954880,16373.999417,0.000099,72.0,11712.532997,31.2500,-0.037374,-0.044556,-5901.227474,10498.046875,-530.133903,26848.0,0.0,2021-06-09 10:55:43.050000115
2021-06-09 10:55:44.046,347.386,0.0,-0.0753,0.2088,0.9751,0.0103,0.1904,0.9816,30.62,-31.73,-32.34,-35.15,-31.50,-29.02,4.79,4.75,4.770,0.5265,0.3164,-26.04,113.69,681.29,65.77,0.04,0.04,0.002488,0.000287,-0.004635,-0.000883,0.003336,-0.004730,0.007158,0.003977,0.000969,-0.005839,0.000580,0.002653,0.005990,-0.000065,-0.004696,-0.006022,0.003245,0.006334,-0.002378,0.000393,0.001642,0.004362,0.000329,0.006240,0.000409,-0.003821,-0.000722,-0.004390,-0.005155,0.001378,0.006062,0.002286,-0.004119,-0.000327,0.004497,0.003090,0.007275,0.003378,-0.004747,-0.004027,-0.002857,0.000155,0.004926,0.004238,0.003966,0.003012,-0.003380,-0.001506,0.001986,-0.000345,0.001374,-0.001365,-0.002922,-0.000590,0.001159,-0.002862,-0.005913,-0.004047,-0.000793,-0.006479,-0.001060,-0.005802,0.002941,-0.005636,-0.052083,-0.052083,-0.052083,-0.052083,-0.002145,37.000731,16319.999471,0.000099,72.0,11709.001541,31.1875,-0.037351,-0.044508,-6208.147526,10539.899826,-530.133903,27347.0,0.0,2021-06-09 10:55:44.048002411
2021-06-09 10:55:45.046,346.418,0.0,-0.1497,0.2569,0.9548,-0.1327,0.1655,0.9772,29.91,-31.48,-32.39,-35.37,-30.83,-28.95,4.48,4.00,4.240,0.6064,0.3539,-48.29,36.95,280.98,65.28,0.48,0.48,0.002660,0.000542,-0.004598,-0.000787,0.003297,-0.004751,0.007097,0.003934,0.000914,-0.005902,0.000612,0.002752,0.005921,-0.000053,-0.004761,-0.006088,0.003288,0.006297,-0.002420,0.000333,0.001650,0.004362,0.000277,0.006197,0.000399,-0.003763,-0.000788,-0.004399,-0.005194,0.001379,0.006046,0.002353,-0.004065,-0.000264,0.004454,0.003058,0.007230,0.003327,-0.004662,-0.003911,-0.002848,0.000186,0.004875,0.004206,0.003913,0.002961,-0.003369,-0.001401,0.001939,-0.000331,0.001299,-0.001424,-0.002818,-0.000377,0.001143,-0.002814,-0.005994,-0.004161,-0.000850,-0.006529,-0.000909,-0.005849,0.002889,-0.005709,-0.052083,-0.052083,-0.052083,-0.052083,-0.002531,37.229978,16290.999949,0.000100,72.0,11712.532997,31.1250,-0.037351,-0.044565,-5998.883724,10498.046875,-530.133903,27846.0,0.0,2021-06-09 10:55:45.046004708
2021-06-09 10:55:53.046,347.109,0.0,0.4306,0.2220,0.8748,0.4931,0.2031,0.8459,33.24,-33.66,-31.23,-32.60,-31.43,-28.04,4.03,4.25,4.140,0.1739,0.2700,385.26,144.29,686.40,65.84,-0.22,0.22,0.002271,0.000115,-0.004653,-0.000792,0.003315,-0.004699,0.007146,0.004011,0.001065,-0.005720,0.000501,0.002668,0.006005,0.000042,-0.004771,-0.006008,0.003296,0.006245,-0.002336,0.00

In [17]:
full_psds_for_2021=pd.read_pickle("full_psds_for_2021.pkl")

In [18]:
full_psds_for_2021

,Fp1-delta ($\mathrm{V^2}$/Hz),Fp2-delta ($\mathrm{V^2}$/Hz),F3-delta ($\mathrm{V^2}$/Hz),F4-delta ($\mathrm{V^2}$/Hz),C3-delta ($\mathrm{V^2}$/Hz),C4-delta ($\mathrm{V^2}$/Hz),P3-delta ($\mathrm{V^2}$/Hz),P4-delta ($\mathrm{V^2}$/Hz),O1-delta ($\mathrm{V^2}$/Hz),O2-delta ($\mathrm{V^2}$/Hz),F7-delta ($\mathrm{V^2}$/Hz),F8-delta ($\mathrm{V^2}$/Hz),T7-delta ($\mathrm{V^2}$/Hz),T8-delta ($\mathrm{V^2}$/Hz),P7-delta ($\mathrm{V^2}$/Hz),P8-delta ($\mathrm{V^2}$/Hz),Fz-delta ($\mathrm{V^2}$/Hz),Cz-delta ($\mathrm{V^2}$/Hz),Pz-delta ($\mathrm{V^2}$/Hz),Oz-delta ($\mathrm{V^2}$/Hz),FC1-delta ($\mathrm{V^2}$/Hz),FC2-delta ($\mathrm{V^2}$/Hz),CP1-delta ($\mathrm{V^2}$/Hz),CP2-delta ($\mathrm{V^2}$/Hz),FC5-delta ($\mathrm{V^2}$/Hz),FC6-delta ($\mathrm{V^2}$/Hz),CP5-delta ($\mathrm{V^2}$/Hz),CP6-delta ($\mathrm{V^2}$/Hz),FT9-delta ($\mathrm{V^2}$/Hz),FT10-delta ($\mathrm{V^2}$/Hz),FCz-delta ($\mathrm{V^2}$/Hz),AFz-delta ($\mathrm{V^2}$/Hz),F1-delta ($\mathrm{V^2}$/Hz),F2-delta ($\mathrm{V^2}$/Hz),C1-delta ($\mathrm{V^2}$/Hz),C2-delta ($\mathrm{V^2}$/Hz),P1-delta ($\mathrm{V^2}$/Hz),P2-delta ($\mathrm{V^2}$/Hz),AF3-delta ($\mathrm{V^2}$/Hz),AF4-delta ($\mathrm{V^2}$/Hz),FC3-delta ($\mathrm{V^2}$/Hz),FC4-delta ($\mathrm{V^2}$/Hz),CP3-delta ($\mathrm{V^2}$/Hz),CP4-delta ($\mathrm{V^2}$/Hz),PO3-delta ($\mathrm{V^2}$/Hz),PO4-delta ($\mathrm{V^2}$/Hz),F5-delta ($\mathrm{V^2}$/Hz),F6-delta ($\mathrm{V^2}$/Hz),C5-delta ($\mathrm{V^2}$/Hz),C6-delta ($\mathrm{V^2}$/Hz),P5-delta ($\mathrm{V^2}$/Hz),P6-delta ($\mathrm{V^2}$/Hz),AF7-delta ($\mathrm{V^2}$/Hz),AF8-delta ($\mathrm{V^2}$/Hz),FT7-delta ($\mathrm{V^2}$/Hz),FT8-delta ($\mathrm{V^2}$/Hz),TP7-delta ($\mathrm{V^2}$/Hz),TP8-delta ($\mathrm{V^2}$/Hz),PO7-delta ($\mathrm{V^2}$/Hz),PO8-delta ($\mathrm{V^2}$/Hz),Fpz-delta ($\mathrm{V^2}$/Hz),CPz-delta ($\mathrm{V^2}$/Hz),POz-delta ($\mathrm{V^2}$/Hz),TP10-delta ($\mathrm{V^2}$/Hz),Fp1-theta ($\mathrm{V^2}$/Hz),Fp2-theta ($\mathrm{V^2}$/Hz),F3-theta ($\mathrm{V^2}$/Hz),F4-theta ($\mathrm{V^2}$/Hz),C3-theta ($\mathrm{V^2}$/Hz),C4-theta ($\mathrm{V^2}$/Hz),P3-theta ($\mathrm{V^2}$/Hz),P4-theta ($\mathrm{V^2}$/Hz),O1-theta ($\mathrm{V^2}$/Hz),O2-theta ($\mathrm{V^2}$/Hz),F7-theta ($\mathrm{V^2}$/Hz),F8-theta ($\mathrm{V^2}$/Hz),T7-theta ($\mathrm{V^2}$/Hz),T8-theta ($\mathrm{V^2}$/Hz),P7-theta ($\mathrm{V^2}$/Hz),P8-theta ($\mathrm{V^2}$/Hz),Fz-theta ($\mathrm{V^2}$/Hz),Cz-theta ($\mathrm{V^2}$/Hz),Pz-theta ($\mathrm{V^2}$/Hz),Oz-theta ($\mathrm{V^2}$/Hz),FC1-theta ($\mathrm{V^2}$/Hz),FC2-theta ($\mathrm{V^2}$/Hz),CP1-theta ($\mathrm{V^2}$/Hz),CP2-theta ($\mathrm{V^2}$/Hz),FC5-theta ($\mathrm{V^2}$/Hz),FC6-theta ($\mathrm{V^2}$/Hz),CP5-theta ($\mathrm{V^2}$/Hz),CP6-theta ($\mathrm{V^2}$/Hz),FT9-theta ($\mathrm{V^2}$/Hz),FT10-theta ($\mathrm{V^2}$/Hz),FCz-theta ($\mathrm{V^2}$/Hz),AFz-theta ($\mathrm{V^2}$/Hz),F1-theta ($\mathrm{V^2}$/Hz),F2-theta ($\mathrm{V^2}$/Hz),C1-theta ($\mathrm{V^2}$/Hz),C2-theta ($\mathrm{V^2}$/Hz),P1-theta ($\mathrm{V^2}$/Hz),P2-theta ($\mathrm{V^2}$/Hz),AF3-theta ($\mathrm{V^2}$/Hz),AF4-theta ($\mathrm{V^2}$/Hz),FC3-theta ($\mathrm{V^2}$/Hz),FC4-theta ($\mathrm{V^2}$/Hz),CP3-theta ($\mathrm{V^2}$/Hz),CP4-theta ($\mathrm{V^2}$/Hz),PO3-theta ($\mathrm{V^2}$/Hz),PO4-theta ($\mathrm{V^2}$/Hz),F5-theta ($\mathrm{V^2}$/Hz),F6-theta ($\mathrm{V^2}$/Hz),C5-theta ($\mathrm{V^2}$/Hz),C6-theta ($\mathrm{V^2}$/Hz),P5-theta ($\mathrm{V^2}$/Hz),P6-theta ($\mathrm{V^2}$/Hz),AF7-theta ($\mathrm{V^2}$/Hz),AF8-theta ($\mathrm{V^2}$/Hz),FT7-theta ($\mathrm{V^2}$/Hz),FT8-theta ($\mathrm{V^2}$/Hz),TP7-theta ($\mathrm{V^2}$/Hz),TP8-theta ($\mathrm{V^2}$/Hz),PO7-theta ($\mathrm{V^2}$/Hz),PO8-theta ($\mathrm{V^2}$/Hz),Fpz-theta ($\mathrm{V^2}$/Hz),CPz-theta ($\mathrm{V^2}$/Hz),POz-theta ($\mathrm{V^2}$/Hz),TP10-theta ($\mathrm{V^2}$/Hz),Fp1-alpha ($\mathrm{V^2}$/Hz),Fp2-alpha ($\mathrm{V^2}$/Hz),F3-alpha ($\mathrm{V^2}$/Hz),F4-alpha ($\mathrm{V^2}$/Hz),C3-alpha ($\mathrm{V^2}$/Hz),C4-alpha ($\mathrm{V^2}$/Hz),P3-alpha ($\mathrm{V^2}$/Hz),P4-alpha (

In [19]:
pd.set_option('display.max_columns', None)
tol_2 = pd.Timedelta('1S')
full_df_2021 = pd.merge_asof(left=full_df, right=full_psds_for_2021, right_index=True, left_index=True, direction='nearest', tolerance=tol_2).dropna()

full_df_2021

,CO2,status_tobii,gd_lx,gd_ly,gd_lz,gd_rx,gd_ry,gd_rz,pc_lx,pc_ly,pc_lz,pc_rx,pc_ry,pc_rz,pd_l,pd_r,pda,gp_x,gp_y,gp3_x,gp3_y,gp3_z,pup_dis,pd_diff,pd_diff_abs,Fp1,Fp2,F3,F4,C3,C4,P3,P4,O1,O2,F7,F8,T7,T8,P7,P8,Fz,Cz,Pz,Oz,FC1,FC2,CP1,CP2,FC5,FC6,CP5,CP6,FT9,FT10,FCz,AFz,F1,F2,C1,C2,P1,P2,AF3,AF4,FC3,FC4,CP3,CP4,PO3,PO4,F5,F6,C5,C6,P5,P6,AF7,AF8,FT7,FT8,TP7,TP8,PO7,PO8,Fpz,CPz,POz,TP10,ExG 1,ExG 2,ExG 3,ExG 4,ECG.,Resp.,PPG,SpO2,HR,GSR,Temp.,AUX 1,AUX 2,ACC77,ACC78,ACC79,Packet Counter,TRIGGER,dateTime,Fp1-delta ($\mathrm{V^2}$/Hz),Fp2-delta ($\mathrm{V^2}$/Hz),F3-delta ($\mathrm{V^2}$/Hz),F4-delta ($\mathrm{V^2}$/Hz),C3-delta ($\mathrm{V^2}$/Hz),C4-delta ($\mathrm{V^2}$/Hz),P3-delta ($\mathrm{V^2}$/Hz),P4-delta ($\mathrm{V^2}$/Hz),O1-delta ($\mathrm{V^2}$/Hz),O2-delta ($\mathrm{V^2}$/Hz),F7-delta ($\mathrm{V^2}$/Hz),F8-delta ($\mathrm{V^2}$/Hz),T7-delta ($\mathrm{V^2}$/Hz),T8-delta ($\mathrm{V^2}$/Hz),P7-delta ($\mathrm{V^2}$/Hz),P8-delta ($\mathrm{V^2}$/Hz),Fz-delta ($\mathrm{V^2}$/Hz),Cz-delta ($\mathrm{V^2}$/Hz),Pz-delta ($\mathrm{V^2}$/Hz),Oz-delta ($\mathrm{V^2}$/Hz),FC1-delta ($\mathrm{V^2}$/Hz),FC2-delta ($\mathrm{V^2}$/Hz),CP1-delta ($\mathrm{V^2}$/Hz),CP2-delta ($\mathrm{V^2}$/Hz),FC5-delta ($\mathrm{V^2}$/Hz),FC6-delta ($\mathrm{V^2}$/Hz),CP5-delta ($\mathrm{V^2}$/Hz),CP6-delta ($\mathrm{V^2}$/Hz),FT9-delta ($\mathrm{V^2}$/Hz),FT10-delta ($\mathrm{V^2}$/Hz),FCz-delta ($\mathrm{V^2}$/Hz),AFz-delta ($\mathrm{V^2}$/Hz),F1-delta ($\mathrm{V^2}$/Hz),F2-delta ($\mathrm{V^2}$/Hz),C1-delta ($\mathrm{V^2}$/Hz),C2-delta ($\mathrm{V^2}$/Hz),P1-delta ($\mathrm{V^2}$/Hz),P2-delta ($\mathrm{V^2}$/Hz),AF3-delta ($\mathrm{V^2}$/Hz),AF4-delta ($\mathrm{V^2}$/Hz),FC3-delta ($\mathrm{V^2}$/Hz),FC4-delta ($\mathrm{V^2}$/Hz),CP3-delta ($\mathrm{V^2}$/Hz),CP4-delta ($\mathrm{V^2}$/Hz),PO3-delta ($\mathrm{V^2}$/Hz),PO4-delta ($\mathrm{V^2}$/Hz),F5-delta ($\mathrm{V^2}$/Hz),F6-delta ($\mathrm{V^2}$/Hz),C5-delta ($\mathrm{V^2}$/Hz),C6-delta ($\mathrm{V^2}$/Hz),P5-delta ($\mathrm{V^2}$/Hz),P6-delta ($\mathrm{V^2}$/Hz),AF7-delta ($\mathrm{V^2}$/Hz),AF8-delta ($\mathrm{V^2}$/Hz),FT7-delta ($\mathrm{V^2}$/Hz),FT8-delta ($\mathrm{V^2}$/Hz),TP7-delta ($\mathrm{V^2}$/Hz),TP8-delta ($\mathrm{V^2}$/Hz),PO7-delta ($\mathrm{V^2}$/Hz),PO8-delta ($\mathrm{V^2}$/Hz),Fpz-delta ($\mathrm{V^2}$/Hz),CPz-delta ($\mathrm{V^2}$/Hz),POz-delta ($\mathrm{V^2}$/Hz),TP10-delta ($\mathrm{V^2}$/Hz),Fp1-theta ($\mathrm{V^2}$/Hz),Fp2-theta ($\mathrm{V^2}$/Hz),F3-theta ($\mathrm{V^2}$/Hz),F4-theta ($\mathrm{V^2}$/Hz),C3-theta ($\mathrm{V^2}$/Hz),C4-theta ($\mathrm{V^2}$/Hz),P3-theta ($\mathrm{V^2}$/Hz),P4-theta ($\mathrm{V^2}$/Hz),O1-theta ($\mathrm{V^2}$/Hz),O2-theta ($\mathrm{V^2}$/Hz),F7-theta ($\mathrm{V^2}$/Hz),F8-theta ($\mathrm{V^2}$/Hz),T7-theta ($\mathrm{V^2}$/Hz),T8-theta ($\mathrm{V^2}$/Hz),P7-theta ($\mathrm{V^2}$/Hz),P8-theta ($\mathrm{V^2}$/Hz),Fz-theta ($\mathrm{V^2}$/Hz),Cz-theta ($\mathrm{V^2}$/Hz),Pz-theta ($\mathrm{V^2}$/Hz),Oz-theta ($\mathrm{V^2}$/Hz),FC1-theta ($\mathrm{V^2}$/Hz),FC2-theta ($\mathrm{V^2}$/Hz),CP1-theta ($\mathrm{V^2}$/Hz),CP2-theta ($\mathrm{V^2}$/Hz),FC5-theta ($\mathrm{V^2}$/Hz),FC6-theta ($\mathrm{V^2}$/Hz),CP5-theta ($\mathrm{V^2}$/Hz),CP6-theta ($\mathrm{V^2}$/Hz),FT9-theta ($\mathrm{V^2}$/Hz),FT10-theta ($\mathrm{V^2}$/Hz),FCz-theta ($\mathrm{V^2}$/Hz),AFz-theta ($\mathrm{V^2}$/Hz),F1-theta ($\mathrm{V^2}$/Hz),F2-theta ($\mathrm{V^2}$/Hz),C1-theta ($\mathrm{V^2}$/Hz),C2-theta ($\mathrm{V^2}$/Hz),P1-theta ($\mathrm{V^2}$/Hz),P2-theta ($\mathrm{V^2}$/Hz),AF3-theta ($\mathrm{V^2}$/Hz),AF4-theta ($\mathrm{V^2}$/Hz),FC3-theta ($\mathrm{V^2}$/Hz),FC4-theta ($\mathrm{V^2}$/Hz),CP3-theta ($\mathrm{V^2}$/Hz),CP4-theta ($\mathrm{V^2}$/Hz),PO3-theta ($\mathrm{V^2}$/Hz),PO4-theta ($\mathrm{V^2}$/Hz),F5-theta ($\mathrm{V^2}$/Hz),F6-theta ($\mathrm{V^2}$/Hz),C5-theta ($\mathrm{V^2}$/Hz),C6-theta ($\mathrm{V^2}$/Hz),P5-theta ($\mathrm{V^2}$/Hz),P6-theta ($\mathrm{V^2}$/Hz),AF7-theta ($\mathrm{V^2}$/Hz),AF8-theta ($\mathrm{V^2}$/Hz),FT7-theta 

In [20]:
#full_df_2021.to_pickle("full_df_2021_CO2.pkl")